In [1]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import os
import json
import numpy as np
import time
import random
import json
from concurrent.futures import ThreadPoolExecutor, as_completed

In [2]:
from glob import glob

files = glob('url/*.json')
urls = []
for f in files:
    with open(f) as fopen:
        urls.extend(json.load(fopen))
urls = sorted(list(set(urls)))
len(urls)

16228

In [3]:
# !rm -rf page
!mkdir page

In [4]:
r = requests.get(urls[0])
soup = BeautifulSoup(r.content, "lxml")

In [5]:
soup.find('h1').text

'&0 / and zero: coly announces “detective” and “romance” game for Spring 2022'

In [6]:
soup.find('div', {'class': 'content-inner'}).text

'\ncoly has announced a new mobile game,\xa0&0 – pronounced\xa0and zero – to be launching in Spring 2022. Much like the company’s previous games such as\xa0Stand My Heroes and\xa0Mahoutsukai no Yakusoku, &0 / and zero will also be targetting the female demographic.\n\n\n\nDubbed a “detective and romance simulation” game, you’ll be playing a heroine who has lost her memory, along the way meeting detectives whose pasts you can gradually unravel and learn about, for a “once in a lifetime” encounter. Presumably this is firmly in the otome game genre rather than the “general” joseimuke umbrella considering the focus on romance.\n\n\n\nRelated Posts\n\n\n\n\n\n\n\nPokemon Sleep Now Available In Malaysia\n\n\n\n\n\nGamerBraves Newsletter Vol. 103 – The Glory of The Barbie/Oppenheimer Double Feature\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nAs there’s no trailer or much information at time of writing, how the gameplay will work out is anyone’s guess. See\xa0Tea

In [7]:
def crawl(url, filename):
    if os.path.exists(filename):
        return
    
    while True:
        try:
            r = requests.get(url)
            break
        except:
            time.sleep(1.0)
    soup = BeautifulSoup(r.content, "lxml")
    
    try:
        title = soup.find('h1').text
    except Exception as e:
        title = None

    try:
        body = soup.find('div', {'class': 'content-inner'}).text
    except Exception as e:
        body = None

    data = {
        'url': url,
        'title': title,
        'body': body,
    }
    with open(filename, 'w') as fopen:
        json.dump(data, fopen)

In [8]:
max_worker = 10

for i in tqdm(range(0, len(urls), max_worker)):
    a = np.arange(i, i + max_worker)
    urls_ = [(urls[k], os.path.join('page', f'{k}.json')) for k in a]
    
    with ThreadPoolExecutor(max_workers=max_worker) as executor:
        futures = {executor.submit(crawl, url[0], url[1]): url for url in urls_}

        for future in as_completed(futures):
            future.result()

100%|████████████████████████████████████▉| 1622/1623 [6:00:47<00:13, 13.35s/it]


IndexError: list index out of range